<a href="https://colab.research.google.com/github/bankmanx9x/python/blob/master/02%20PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update                                                                          # อัพเดท Package ทั้งหมดใน VM ตัวนี้
!apt-get install openjdk-8-jdk-headless -qq > /dev/null                                  # ติดตั้ง Java Development Kit (จำเป็นสำหรับการติดตั้ง Spark)
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz # ติดตั้ง Spark 3.1.2
!tar xzvf spark-3.1.2-bin-hadoop2.7.tgz                                                  # Unzip ไฟล์ Spark 3.1.2
!pip install -q findspark==1.3.0                                                         # ติดตั้ง Package Python สำหรับเชื่อมต่อกับ Spark 

In [ ]:
# Set enviroment variable ให้ Python รู้จัก Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
# ติดตั้ง PySpark ลงใน Python
!pip install pyspark==3.1.2

In [ ]:
# Server ของ Google Colab มีกี่ Core
!cat /proc/cpuinfo

In [ ]:
# สร้าง Spark Session เพิ้อใช้งาน Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# สร้าง Spark Session เพิ้อใช้งาน Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# ดูเวอร์ชั่น Spark
spark.version

'3.1.2'

คำอธิบายคำสั่ง:

wget = คำสั่งในการดาวน์โหลดไฟล์

wget -O = ตั้งชื่อไฟล์

In [ ]:
# Download Data File
!wget -O data.zip https://file.designil.com/zdOfUE+
!unzip data.zip

In [ ]:
dt = spark.read.csv('/content/ws2_data.csv', header = True, inferSchema = True, )

# Step 2) Data Profiling

In [ ]:
# ดูว่ามีคอลัมน์อะไรบ้าง
dt

DataFrame[timestamp: string, user_id: string, book_id: int, country: string, price: double]

In [ ]:
# ดูข้อมูล
dt.show()

In [ ]:
# ดูข้อมูล
dt.show()

In [ ]:
# ดูประเภทข้อมูลแต่ละคอลัมน์
dt.dtypes

In [ ]:
# อีกคำสั่งในการดูข้อมูลแต่ละคอลัมน์ (Schema)
dt.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- price: double (nullable = true)



In [ ]:
# นับจำนวนแถวและ column
print((dt.count(), len(dt.columns)))

(500000, 5)


In [ ]:
# สรุปข้อมูลสถิติ
dt.describe().show()

+-------+-------------------+--------+-----------------+-------+------------------+
|summary|          timestamp| user_id|          book_id|country|             price|
+-------+-------------------+--------+-----------------+-------+------------------+
|  count|             500000|  499998|           500000| 500000|            500000|
|   mean|               null|Infinity|       740.377786|   null|31.459311680039008|
| stddev|               null|     NaN|559.9366172146341|   null|10.945931459891705|
|    min|2021-05-01 00:00:04|00000f49|                1|Algeria|               0.0|
|    max|2021-07-31 23:59:55|ffffe3e7|             2269|  Yemen|              84.0|
+-------+-------------------+--------+-----------------+-------+------------------+



In [ ]:
# อีกคำสั่งในการสรุปข้อมูลสถิติ
# ReferenceL: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.summary.html

dt.summary().show()

+-------+-------------------+-----------+-----------------+-------+------------------+
|summary|          timestamp|    user_id|          book_id|country|             price|
+-------+-------------------+-----------+-----------------+-------+------------------+
|  count|             500000|     499998|           500000| 500000|            500000|
|   mean|               null|   Infinity|       740.377786|   null|31.459311680039008|
| stddev|               null|        NaN|559.9366172146341|   null|10.945931459891705|
|    min|2021-05-01 00:00:04|   00000f49|                1|Algeria|               0.0|
|    25%|               null|2.4050123E7|              262|   null|             24.95|
|    50%|               null|  7.04742E7|              614|   null|             29.99|
|    75%|               null|  5.5726E81|             1217|   null|             34.99|
|    max|2021-07-31 23:59:55|   ffffe3e7|             2269|  Yemen|              84.0|
+-------+-------------------+-----------+--

In [ ]:
# สรุปข้อมูลสถิติเฉพาะ column ที่ระบุ
dt.select("price").describe().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            500000|
|   mean|31.459311680039008|
| stddev|10.945931459891705|
|    min|               0.0|
|    max|              84.0|
+-------+------------------+



### Exercise 1

คอลัมน์ไหนมี Missing Value บ้าง? และแสดงข้อมูลแถวที่มี Missing Value ให้ดูหน่อย

In [ ]:
dt.where(dt.user_id.isNull()).show()

+-------------------+-------+-------+-------+-----+
|          timestamp|user_id|book_id|country|price|
+-------------------+-------+-------+-------+-----+
|2021-05-01 14:41:20|   null|    516| Brazil|27.37|
|2021-06-30 02:27:00|   null|   1390|  Japan|27.99|
+-------------------+-------+-------+-------+-----+



In [ ]:
# ข้อมูลที่เป็นตัวเลข
dt.where(dt.price >= 1).show()

In [ ]:
# ข้อมูลที่เป็นตัวหนังสือ
dt.where(dt.country == 'Canada').show()

In [ ]:
dt.show(5)

+-------------------+--------+-------+------------+-----+
|          timestamp| user_id|book_id|     country|price|
+-------------------+--------+-------+------------+-----+
|2021-05-01 08:03:44|824dc295|      2|      Israel|24.95|
|2021-05-02 02:48:34|f80fe106|      6|    Thailand|31.49|
|2021-05-01 16:54:57|01cb9fe0|      9|       Italy|29.65|
|2021-05-01 21:45:28|59508f47|     13|South Africa| 22.4|
|2021-05-01 14:50:52|3553ebbb|     14|      Canada| 28.0|
+-------------------+--------+-------+------------+-----+
only showing top 5 rows



In [ ]:
dt.where(dt.timestamp.startswith('2021-04-01')).show()

+---------+-------+-------+-------+-----+
|timestamp|user_id|book_id|country|price|
+---------+-------+-------+-------+-----+
+---------+-------+-------+-------+-----+

